---

title: 输入事件设备
keywords: Maixpy3, 输入事件 event
desc: maixpy3 doc: 输入事件 event

---

| 更新时间 | 负责人 | 内容 | 备注 |
| --- | --- | --- | --- |
| 2022年1月4日 | Rui | 初次编写文档 | --- |
| 2022年1月8日 | Rui | 修改文档的编写方式 | 使用 Jupyter notebook 进行编写文档 |
| 2022年1月18日 | Rui | 修改文档，增加效果图 | 外设文档通过测试的平台有 MaixII-Dock，使用的是 MaixPy3 0.4.0 |

输入事件是 Linux 系统中都存在的一种特殊设备（/dev/event/input），可以通过事件来检测外接的鼠标、键盘等设备是否发生变化，如果检测键盘输入了什么进行了什么样的操作，一样可以通过输入事件来获取。

## 使用方法

### 准备

接上事件设备，例如树莓派，接上键盘鼠标等事件检测设备。对于 MaixII-Dock，开发板上的按键是两个事件设备
![111](./../asserts/M2Dock_pin.jpg)


### 代码

下面是在 MaixII-Dock 上运行的代码，用于检测两个按键的输入。如果外接别的设备，这需要修改 `event.InputDevice()` 中的参数。

In [1]:
from maix import event
from select import select

def check_key():
	import os
	tmp = "/dev/input/by-path/"
	if os.path.exists(tmp):
		for i in os.listdir(tmp):
			if i.find("kbd") != -1:
				return tmp + i
	return "/dev/input/event0"

count = 0
dev = event.InputDevice(check_key())
while True:
	r, w, x = select([dev], [], [], 0) # if r == 0 or set 0 will read() raise BlockingIOError 
	if r:
		for data in dev.read():
			print(data)
			if data.code == 0x02:
				print('press key S1')
			if data.code == 0x03:
				print('press key S2')
			if data.value == 1 and data.code != 0:
				count += 1
		print('press sum:', count)


[ rpyc-kernel ]( running at Mon Jan 17 18:30:27 2022 )
press key S1
press sum: 1
press key S1
press key S2
press sum: 2
press key S2
press key S1
press sum: 3
press key S1
press key S2
press sum: 4
press key S2
press key S1
press sum: 5
press key S1
press key S2
press sum: 6
press key S2



Traceback (most recent call last):
  File "<string>", line unknown, in <module>
Remote.KeyboardInterrupt


运行代码之后，按下按键则会有内容打印出来

通过 `/dev/input/event0` 进行事件设备的选择，可以通过 `os.system("ls /dev/input/")` 进行查看接入了多少事件设备

> 详细请看 [python-evdev](https://python-evdev.readthedocs.io/) 或了解底层 linux evdev 输入子系统。